Hello Dears, in this code you can fine the ELMo text embbeder, which use a bidirectional language model.
This code contains 12 ML models to improve the contextual analysis of the dataset

---





### ELMO Word emmbeding 

In [0]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [4]:
# read data
train = pd.read_csv("/content/drive/My Drive/Third Semester/Machine Learning /MLProject/Elmo/train.csv", encoding='utf-8')
test = pd.read_csv("/content/drive/My Drive/Third Semester/Machine Learning /MLProject/Elmo/test.csv", encoding='utf-8')

train.shape, test.shape

((9652, 3), (2419, 2))

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
train['label'].value_counts(normalize = True)

0.800000    0.122876
0.600000    0.120804
1.000000    0.119664
0.400000    0.113966
1.200000    0.105367
0.200000    0.096146
1.400000    0.084853
1.600000    0.065789
0.000000    0.054186
1.800000    0.044550
2.000000    0.031910
2.200000    0.017302
2.400000    0.009739
2.600000    0.003937
2.800000    0.001347
1.300000    0.001347
1.100000    0.001140
1.500000    0.000725
1.700000    0.000725
0.900000    0.000725
1.900000    0.000518
0.700000    0.000414
0.500000    0.000414
2.100000    0.000311
2.300000    0.000207
3.000000    0.000207
0.300000    0.000207
2.133333    0.000104
1.533333    0.000104
1.333333    0.000104
2.500000    0.000104
0.866667    0.000104
2.066667    0.000104
Name: label, dtype: float64

In [0]:
# remove URL's from train and test
train['clean_tweet'] = train['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

test['clean_tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

In [0]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# convert text to lowercase
train['clean_tweet'] = train['clean_tweet'].str.lower()
test['clean_tweet'] = test['clean_tweet'].str.lower()

# remove numbers
train['clean_tweet'] = train['clean_tweet'].str.replace("[0-9]", " ")
test['clean_tweet'] = test['clean_tweet'].str.replace("[0-9]", " ")

# remove whitespaces
train['clean_tweet'] = train['clean_tweet'].apply(lambda x:' '.join(x.split()))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ' '.join(x.split()))

In [0]:
# import spacy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [0]:
train['clean_tweet'] = lemmatization(train['clean_tweet'])
test['clean_tweet'] = lemmatization(test['clean_tweet'])

In [14]:
train.sample(10)

,id,label,tweet,clean_tweet
1370,7360,1.4,world largest collect presidenti garbag twice size texa,world large collect presidenti garbag twice size texa
6401,8842,0.6,republican healthcar bill actual emerg ash almost certainli chang senat,republican healthcar bill actual emerg ash almost certainli chang senat
9526,11870,1.6,pitbul see true hair puerto rico relief,pitbul see true hair puerto rico relief
4257,12835,0.4,malinger rail republican obamacar rebel bbc news,malinger rail republican obamacar rebel bbc news
3626,5832,0.6,tiffani went playboy parti year eve internet explod,tiffani go playboy parti year eve internet explod
3310,11189,1.0,soda drought south african citi may avoid day zero,soda drought south african citi may avoid day zero
513,14071,0.0,husband pac man japanes arcad pioneer masaya nakamura die 91,husband pac man japan arcad pioneer masaya nakamura die
3561,8809,0.2,man destroy razorback statu arkansa capitol,man destroy razorback statu arkansa capitol
740,5485,1.0,linguist analysi found danc third grade level,linguist analysi find danc third grade level
4842,5194,0.0,tillerson tweet report fire twitter,tillerson tweet report fire twitter


In [16]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [17]:
# just a random sentence
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

embeddings.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


TensorShape([Dimension(1), Dimension(8), Dimension(1024)])

In [0]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [0]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [0]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['clean_tweet']) for x in list_train]
elmo_test = [elmo_vectors(x['clean_tweet']) for x in list_test]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Pretrain preperation


In [0]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [0]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [0]:
# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

# load elmo_train_new
pickle_in = open("elmo_test_03032019.pickle", "rb")
elmo_test_new = pickle.load(pickle_in)

**Start the ML **

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict

In [0]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  train['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)

In [0]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
rf = RandomForestRegressor( max_depth=5, verbose=3, n_estimators=100)
parameters = {'n_estimators': [120, 100], 'max_depth':[3,5,]}
grid = GridSearchCV(rf, parameters, cv=3, n_jobs=-1, verbose=3, scoring=make_scorer(mean_squared_error))

In [0]:
'''# Ranking Features for train data
rnd_clf_reg = RandomForestRegressor(n_estimators=100, n_jobs=-1)
rnd_clf_reg.fit(xtrain, ytrain)
importances = pd.DataFrame({'feature': X_scaled_train.columns, 'importance': np.round(rnd_clf_reg.feature_importances_, 3)})
importances = importances.sort_values('importance', ascending=False).set_index('feature')
print("Rank Features :")
importances.head(22)'''

'# Ranking Features for train data\nrnd_clf_reg = RandomForestRegressor(n_estimators=100, n_jobs=-1)\nrnd_clf_reg.fit(xtrain, ytrain)\nimportances = pd.DataFrame({\'feature\': X_scaled_train.columns, \'importance\': np.round(rnd_clf_reg.feature_importances_, 3)})\nimportances = importances.sort_values(\'importance\', ascending=False).set_index(\'feature\')\nprint("Rank Features :")\nimportances.head(22)'

In [0]:
#XGBoost
#0.56841 (25)	
import xgboost as xgb

xgb_reg_model = xgb.XGBRegressor()

xgb_reg_model.fit(xtrain, ytrain)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:41:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
#highest
#NaiveBayes
#0.56502 (24)
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
NB_reg = linear_model.BayesianRidge(n_iter=600)

NB_reg.fit(xtrain, ytrain)

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
              fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=600,
              normalize=False, tol=0.001, verbose=False)

In [0]:
#0.5650
from sklearn.ensemble import GradientBoostingRegressor

GB = GradientBoostingRegressor(min_samples_leaf= 3, max_depth= 4, learning_rate= 0.1)
GB.fit(xtrain, ytrain)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=3, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
#using the mlp 
#0.5650
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500)
mlp.fit(xtrain, ytrain)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(8, 8, 8), learning_rate='constant',
             learning_rate_init=0.001, max_iter=500, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
#0.5655
# linear SVM in regression
from sklearn.svm import SVR
regressor_SVM = SVR(kernel='rbf')
regressor_SVM.fit(xtrain, ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
#0.60033
#Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
random_forest_reg = RandomForestRegressor()
random_forest_reg.fit(xtrain, ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [0]:
#worst
#KNeighborsRegressor
#0.7227
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=2)
knn.fit(xtrain, ytrain)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                    weights='uniform')

In [0]:
#DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

DT = DecisionTreeRegressor(random_state = 0, max_depth=22) 
DT.fit(xtrain, ytrain)

In [0]:
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

model_lgb.fit(xtrain, ytrain)

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=9,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.2319, feature_fraction_seed=9,
              importance_type='split', learning_rate=0.05, max_bin=55,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_data_in_leaf=6, min_split_gain=0.0,
              min_sum_hessian_in_leaf=11, n_estimators=720, n_jobs=-1,
              num_leaves=5, objective='regression', random_state=None,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [0]:
#0.7227
#ENet 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC

ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

ENet.fit(xtrain, ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17.971878051757812, tolerance: 0.2632855772972107
  positive)


Pipeline(memory=None,
         steps=[('robustscaler',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)),
                ('elasticnet',
                 ElasticNet(alpha=0.0005, copy_X=True, fit_intercept=True,
                            l1_ratio=0.9, max_iter=1000, normalize=False,
                            positive=False, precompute=False, random_state=3,
                            selection='cyclic', tol=0.0001,
                            warm_start=False))],
         verbose=False)

In [0]:
#lasso
#0.7227
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

lasso.fit(xtrain, ytrain)

NameError: ignored

In [0]:
#0.7227
from sklearn.kernel_ridge import KernelRidge

KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
KRR.fit(xtrain, ytrain)

KernelRidge(alpha=0.6, coef0=2.5, degree=2, gamma=None, kernel='polynomial',
            kernel_params=None)

In [0]:
'''parameters = {'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
             'max_features' : ["auto", "sqrt", "log2"],
             'random_state': [0, 25, 75, 125, 250],
             'min_samples_leaf': [2, 3, 4, 5, 6, 7, 8, 9, 10]}
clf = GridSearchCV(DT, parameters)
clf.fit(X_train, y_train)

DT_Model = clf.best_estimator_
print ( clf.best_params_)
print(DT_Model)'''

'parameters = {\'max_depth\': [2, 3, 4, 5, 6, 7, 8, 9, 10],\n             \'max_features\' : ["auto", "sqrt", "log2"],\n             \'random_state\': [0, 25, 75, 125, 250],\n             \'min_samples_leaf\': [2, 3, 4, 5, 6, 7, 8, 9, 10]}\nclf = GridSearchCV(DT, parameters)\nclf.fit(X_train, y_train)\n\nDT_Model = clf.best_estimator_\nprint ( clf.best_params_)\nprint(DT_Model)'

In [0]:
#Done
preds_valid = NB_reg.predict(xvalid)

In [0]:
#Done
preds_valid = random_forest_reg.predict(xvalid)

In [0]:
#Done
preds_valid = regressor_SVM.predict(xvalid)

In [0]:
#Done
preds_valid = xgb_reg_model.predict(xvalid)

In [0]:
#Done
preds_valid = GB.predict(xvalid)

In [0]:
#Done
preds_valid = mlp.predict(xvalid)

In [0]:
#Done
preds_valid = random_forest_reg.predict(xvalid)

In [0]:
# Done
preds_valid = knn.predict(xvalid)

In [0]:
preds_valid = DecisionTreeRegressor.predict(xvalid)

In [0]:
#Done
preds_valid = model_lgb.predict(xvalid)

In [0]:
#Done
preds_valid = ENet.predict(xvalid)

In [0]:
#Done
preds_valid = lasso.predict(xvalid)

In [0]:
#the test new elmo file
preds_test = NB_reg.predict(elmo_test_new)

In [0]:
# prepare submission dataframe
sub = pd.DataFrame({'id':test['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv("/content/drive/My Drive/Third Semester/Machine Learning /MLProject/Elmo/task-1-outputNB_reg.csv", index=False)